In [1]:
import gym
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import sys
sys.path.append("../")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from mbt_gym.gym.TradingEnvironment import TradingEnvironment
from mbt_gym.gym.Traders import AmmTrader

In [5]:
from mbt_gym.agents.BaselineAgents import CarteaJaimungalMmAgent
from mbt_gym.gym.helpers.generate_trajectory import generate_trajectory
from mbt_gym.gym.StableBaselinesTradingEnvironment import StableBaselinesTradingEnvironment
from mbt_gym.gym.wrappers import *
from mbt_gym.rewards.RewardFunctions import CjCriterion, CjMmCriterion
from mbt_gym.stochastic_processes.midprice_models import *
from mbt_gym.stochastic_processes.fill_probability_models import *
from mbt_gym.stochastic_processes.arrival_models import *

In [6]:
terminal_time = 1.0
arrival_rate = 10.0
n_steps = int(10 * terminal_time * arrival_rate)
phi = 0.5
alpha = 0.001

num_trajectories = 3

In [7]:
def get_amm_env_Poisson(num_trajectories:int = 1):    
    fill_exponent = 1
    sigma = 0.001
    initial_inventory = 1_000
    initial_price = 100
    unit_size = 1.0
    max_depth = 5*1/fill_exponent
    step_size = terminal_time/n_steps
    midprice_model = AmmSelfContainedMidpriceModel(jump_size_L=1.0, 
                                                   unit_size = unit_size,
                                                   terminal_time=terminal_time, 
                                                       step_size=step_size, 
                                                       initial_price=initial_price, 
                                                       num_trajectories=num_trajectories)
    arrival_model = PoissonArrivalModel(intensity=np.array([arrival_rate, arrival_rate]), step_size=step_size)
    fill_probability_model = ExponentialFillFunction(fill_exponent=fill_exponent, 
                                                                       step_size=step_size, 
                                                                       num_trajectories=num_trajectories)
    AMMtrader = AmmTrader(midprice_model = midprice_model, arrival_model = arrival_model, 
                                fill_probability_model = fill_probability_model,
                                num_trajectories = num_trajectories, max_depth = max_depth, unit_size = unit_size)
    
    env_params = dict(terminal_time=terminal_time, 
                      n_steps=n_steps,
                      initial_inventory = initial_inventory,
                      midprice_model = midprice_model,
                      arrival_model = arrival_model,
                      fill_probability_model = fill_probability_model,
                      trader = AMMtrader,
                      reward_function = CjMmCriterion(phi, alpha),
                      normalise_action_space = False,
                      normalise_observation_space = False,
                      num_trajectories=num_trajectories)
    return TradingEnvironment(**env_params)

In [8]:
env = get_amm_env_Poisson(num_trajectories=num_trajectories)

In [9]:
env.trader.midprice_model

In [10]:
env.reset()

array([[   0., 1000.,    0.,  100.],
       [   0., 1000.,    0.,  100.],
       [   0., 1000.,    0.,  100.]])

CASH_INDEX = 0
INVENTORY_INDEX = 1
TIME_INDEX = 2
ASSET_PRICE_INDEX = 3

In [11]:
env.state

array([[   0., 1000.,    0.,  100.],
       [   0., 1000.,    0.,  100.],
       [   0., 1000.,    0.,  100.]])

In [12]:
env.arrival_model.intensity

array([10., 10.])

In [13]:
env.arrival_model.get_arrivals()

array([[False, False],
       [False, False],
       [False, False]])

In [ ]:
action = np.array([[0.2,0.2], 
                    [0.2,0.2], 
                     [0.2,0.2]])
env.step(action = action)

> /Users/leandro/Documents/GitHub/amm_gym/mbt_gym/stochastic_processes/midprice_models.py(457)update()
    455         pdb.set_trace()
    456 
--> 457         eta_ask = self.unit_size /( self.jump_size_L  * (previous_inventory - self.unit_size))
    458         eta_bid = self.unit_size /( self.jump_size_L  * (previous_inventory + self.unit_size))
    459         self.current_state = (

ipdb> n!
> /Users/leandro/Documents/GitHub/amm_gym/mbt_gym/stochastic_processes/midprice_models.py(458)update()
    456 
    457         eta_ask = self.unit_size /( self.jump_size_L  * (previous_inventory - self.unit_size))
--> 458         eta_bid = self.unit_size /( self.jump_size_L  * (previous_inventory + self.unit_size))
    459         self.current_state = (
    460             self.current_state

ipdb> n!
> /Users/leandro/Documents/GitHub/amm_gym/mbt_gym/stochastic_processes/midprice_models.py(460)update()
    458         eta_bid = self.unit_size /( self.jump_size_L  * (previous_inventory + self.u